In [9]:
# package import statement
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger
import time
from datetime import datetime, timedelta
import pyotp
import pandas as pd

In [10]:
from datetime import datetime, timedelta
import pandas as pd
import time

In [11]:
import time
from datetime import datetime, timedelta
import pyotp
import pandas as pd
import win32com.client
from SmartApi import SmartConnect
from logzero import logger

In [13]:
api_key = 'FYBFTPOl'
username = 'AAAF620196'
pwd = '0054'

smartApi = SmartConnect(api_key)
try:
    token = "YS4MOR7OUZEHGNV3VFOQ5TVMRE" 
    totp = pyotp.TOTP(token).now()
except Exception as e:
    logger.error("Invalid Token: The provided token is not valid.")
    raise e

correlation_id = "abcde"
data = smartApi.generateSession(username, pwd, totp)

if data['status'] == False:
    logger.error(data)
    
else:
    # login api call
    # logger.info(f"You Credentials: {data}")
    authToken = data['data']['jwtToken']
    refreshToken = data['data']['refreshToken']
    # fetch the feedtoken
    feedToken = smartApi.getfeedToken()
    # fetch User Profile
    res = smartApi.getProfile(refreshToken)
    smartApi.generateToken(refreshToken)
    res=res['data']['exchanges']

[I 250603 12:48:38 smartConnect:124] in pool


In [14]:
symbol_token_map = {
    "HDFCBANK": "1333", "ICICIBANK": "4963", "AXISBANK": "5900", "KOTAKBANK": "1922",
    "SBIN": "3045", "INDUSINDBK": "5258", "BANDHANBNK": "2263", "PNB": "10666",
    "IDFCFIRSTB": "11184", "AUBANK": "21238", "FEDERALBNK": "1023", "BANKBARODA": "4668"
}
bank_nifty_stocks = list(symbol_token_map.keys())
file_path = r"C:\\Users\\jaiad\\Pro T\\banknifty_dashboard.xlsx"

In [15]:
weight_map = {}  # Global variable to store weightages
last_weightage_fetch = None

# Get weightages from smart-investing
def get_banknifty_weightage():
    global weight_map, last_weightage_fetch
    if last_weightage_fetch == datetime.now().date():
        return  # Already fetched today

    url = "https://www.smart-investing.in/indices-bse-nse.php?index=BANKNIFTY"
    try:
        tables = pd.read_html(url)
        for table in tables:
            cols = [col.lower() for col in table.columns.astype(str)]
            if "company" in cols and any("weight" in c for c in cols):
                table.columns = [col.strip() for col in table.columns]
                table = table.rename(columns={
                    table.columns[0]: "Bank Name",
                    table.columns[1]: "Weightage (%)"
                })
                table["Weightage (%)"] = table["Weightage (%)"].str.replace("%", "").astype(float)

                alias_map = {
                    "HDFC BANK": "HDFCBANK",
                    "ICICI BANK": "ICICIBANK",
                    "AXIS BANK": "AXISBANK",
                    "KOTAK MAHINDRA BANK": "KOTAKBANK",
                    "STATE BANK OF INDIA": "SBIN",
                    "INDUSIND BANK": "INDUSINDBK",
                    "BANDHAN BANK": "BANDHANBNK",
                    "PUNJAB NATIONAL BANK": "PNB",
                    "IDFC FIRST BANK": "IDFCFIRSTB",
                    "AU SMALL FINANCE BANK": "AUBANK",
                    "THE FEDERAL BANK": "FEDERALBNK",
                    "BANK OF BARODA": "BANKBARODA"
                }

                weight_map.clear()
                for _, row in table.iterrows():
                    full_name = row["Bank Name"].upper().replace(" LTD", "").strip()
                    symbol = alias_map.get(full_name, None)
                    if symbol:
                        weight_map[symbol] = row["Weightage (%)"]
                last_weightage_fetch = datetime.now().date()
                return
        print("Weightage table not found")
    except Exception as e:
        print("Error fetching weightage:", e)

# Excel init
def initialize_excel():
    excel = win32com.client.Dispatch("Excel.Application")
    excel.Visible = True
    for wb in excel.Workbooks:
        if wb.FullName.lower() == file_path.lower():
            return wb, wb.Worksheets(1)
    try:
        wb = excel.Workbooks.Open(file_path)
        return wb, wb.Worksheets(1)
    except:
        wb = excel.Workbooks.Add()
        sheet = wb.Worksheets(1)
        headers = ["Bank", "LTP", "Volume", "Change %", "Status", "Weightage"]
        for col, header in enumerate(headers, 1):
            sheet.Cells(1, col).Value = header
        wb.SaveAs(file_path)
        return wb, sheet


In [16]:
# Compute status from price change
def get_status(change):
    if change > 0:
        return "Buying"
    elif change < 0:
        return "Selling"
    return "Neutral"

def update_excel(workbook, sheet):
    def get_last_trading_day(current_date):
            last_trading_day = current_date - timedelta(days=1)
            # If today is Monday, the last trading day is Friday
            if current_date.weekday() == 0:  # Monday
                last_trading_day = current_date - timedelta(days=3)
            # If today is Sunday, the last trading day is Friday
            elif current_date.weekday() == 6:  # Sunday
                last_trading_day = current_date - timedelta(days=2)
            # If today is Saturday, the last trading day is Friday
            elif current_date.weekday() == 5:  # Saturday
                last_trading_day = current_date - timedelta(days=1)
            return last_trading_day
    results = []
    now = datetime.now()
    today_open = now.replace(hour=9, minute=15, second=0, microsecond=0)
    market_close = now.replace(hour=15, minute=30, second=0, microsecond=0)
    
    # Check if today is Saturday (weekday == 5) or Sunday (weekday == 6)
    if now.weekday() >= 5:
        print("⚠️ Market is closed on Saturday and Sunday.")
        # Fetch last traded OHLC data (from the last trading day, typically Friday)
        last_trading_day = get_last_trading_day(now)
        from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
        to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
    else:
        if now < today_open:
            print("⚠️ Market has not opened yet. Try again after 9:15 AM.")
            # Fetch last traded OHLC data (from the previous trading day)
            last_trading_day = get_last_trading_day(now)
            from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
            to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
        elif now > market_close:
            print("⚠️ Market is closed for the day.")
            # Fetch last traded OHLC data (from today's market close)
            from_date = (market_close - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
            to_date = market_close.strftime("%Y-%m-%d %H:%M")
        else:
            # Market is open, fetch live data
            from_date = (now - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
            to_date = now.strftime("%Y-%m-%d %H:%M")

    get_banknifty_weightage()  # Fetch weightage data
    sheet.UsedRange.Clear()
    headers = ["Bank", "LTP", "Volume", "Change %", "Status", "Weightage"]
    for col, header in enumerate(headers, 1):
        sheet.Cells(1, col).Value = header

    row_index = 2
    for stock in bank_nifty_stocks:
        try:
            histParam = {
                "exchange": "NSE",
                "symboltoken": symbol_token_map[stock],
                "interval": "ONE_MINUTE",
                "fromdate": from_date,
                "todate": to_date
            }
            response = smartApi.getCandleData(histParam)
            if response['status'] and response['data']:
                candle = response['data'][-1]
                ltp = candle[4]
                volume = candle[5]
                change_percent = ((candle[4] - candle[1]) / candle[1]) * 100
                status = get_status(change_percent)
                weight = weight_map.get(stock.upper(), "")
                sheet.Cells(row_index, 1).Value = stock
                sheet.Cells(row_index, 2).Value = ltp
                sheet.Cells(row_index, 3).Value = volume
                sheet.Cells(row_index, 4).Value = round(change_percent, 2)
                sheet.Cells(row_index, 5).Value = status
                sheet.Cells(row_index, 6).Value = weight
                row_index += 1
            else:
                print(f"⚠️ Failed to get {stock} ONE_MINUTE data:", response)
        except Exception as e:
            print(f"❌ Error fetching {stock} data:", e)
    
    sheet.Cells(row_index+1, 1).Value = "Data Fetched At"
    sheet.Cells(row_index+1, 2).Value = to_date
    
    workbook.Save()
    print(f"✅ Bank Nifty data updated in Excel at {datetime.now().strftime('%H:%M:%S')}")

In [ ]:
try:
    wb, sheet = initialize_excel()
    if wb:
        while True:
            update_excel(wb, sheet)
            time.sleep(60)
except KeyboardInterrupt:
    print("Stopped by user.")
except Exception as e:
    print("Unexpected error:", e)

[E 250603 12:48:58 smartConnect:246] Error occurred while making a POST request to https://apiconnect.angelone.in/rest/secure/angelbroking/historical/v1/getCandleData. Error: Something Went Wrong, Please Try After Sometime. URL: https://apiconnect.angelone.in/rest/secure/angelbroking/historical/v1/getCandleData, Headers: {'Content-type': 'application/json', 'X-ClientLocalIP': '127.0.0.1', 'X-ClientPublicIP': '106.193.147.98', 'X-MACAddress': 'eb:f2:cf:fa:da:c0', 'Accept': 'application/json', 'X-PrivateKey': 'FYBFTPOl', 'X-UserType': 'USER', 'X-SourceID': 'WEB'}, Request: {'exchange': 'NSE', 'symboltoken': '1333', 'interval': 'ONE_MINUTE', 'fromdate': '2025-06-03 11:48', 'todate': '2025-06-03 12:48'}, Response: {'message': 'Something Went Wrong, Please Try After Sometime', 'errorcode': 'AB1004', 'status': False, 'data': None}


⚠️ Failed to get HDFCBANK ONE_MINUTE data: {'message': 'Something Went Wrong, Please Try After Sometime', 'errorcode': 'AB1004', 'status': False, 'data': None}
✅ Bank Nifty data updated in Excel at 12:49:12
✅ Bank Nifty data updated in Excel at 12:50:28


Final placement in xcel : 

In [1]:
import time
from datetime import datetime, timedelta
import pyotp
import pandas as pd
import win32com.client
from SmartApi import SmartConnect
from logzero import logger
import os

class BankNiftyDashboard:
    def __init__(self, api_key, username, pwd, token, file_path, offset_rows=0):
        self.api_key = api_key
        self.username = username
        self.pwd = pwd
        self.token = token
        self.file_path = file_path
        self.offset_rows = offset_rows

        self.symbol_token_map = {
            "HDFCBANK": "1333", "ICICIBANK": "4963", "AXISBANK": "5900", "KOTAKBANK": "1922",
            "SBIN": "3045", "INDUSINDBK": "5258", "BANDHANBNK": "2263", "PNB": "10666",
            "IDFCFIRSTB": "11184", "AUBANK": "21238", "FEDERALBNK": "1023", "BANKBARODA": "4668"
        }
        self.bank_nifty_stocks = list(self.symbol_token_map.keys())
        self.weight_map = {}
        self.last_weightage_fetch = None

        self.smartApi = SmartConnect(self.api_key)
        self.authenticate()
        self.wb, self.sheet = self.initialize_excel()

    def authenticate(self):
        try:
            totp = pyotp.TOTP(self.token).now()
        except Exception as e:
            logger.error("Invalid Token: The provided token is not valid.")
            raise e

        data = self.smartApi.generateSession(self.username, self.pwd, totp)
        if data['status'] == False:
            logger.error(data)
            raise Exception("Authentication failed")
        else:
            self.authToken = data['data']['jwtToken']
            self.refreshToken = data['data']['refreshToken']
            self.feedToken = self.smartApi.getfeedToken()
            self.profile = self.smartApi.getProfile(self.refreshToken)
            self.smartApi.generateToken(self.refreshToken)

    def get_banknifty_weightage(self):
        if self.last_weightage_fetch == datetime.now().date():
            return

        url = "https://www.smart-investing.in/indices-bse-nse.php?index=BANKNIFTY"
        try:
            tables = pd.read_html(url)
            for table in tables:
                cols = [col.lower() for col in table.columns.astype(str)]
                if "company" in cols and any("weight" in c for c in cols):
                    table.columns = [col.strip() for col in table.columns]
                    table = table.rename(columns={
                        table.columns[0]: "Bank Name",
                        table.columns[1]: "Weightage (%)"
                    })
                    table["Weightage (%)"] = table["Weightage (%)"].str.replace("%", "").astype(float)

                    alias_map = {
                        "HDFC BANK": "HDFCBANK",
                        "ICICI BANK": "ICICIBANK",
                        "AXIS BANK": "AXISBANK",
                        "KOTAK MAHINDRA BANK": "KOTAKBANK",
                        "STATE BANK OF INDIA": "SBIN",
                        "INDUSIND BANK": "INDUSINDBK",
                        "BANDHAN BANK": "BANDHANBNK",
                        "PUNJAB NATIONAL BANK": "PNB",
                        "IDFC FIRST BANK": "IDFCFIRSTB",
                        "AU SMALL FINANCE BANK": "AUBANK",
                        "THE FEDERAL BANK": "FEDERALBNK",
                        "BANK OF BARODA": "BANKBARODA"
                    }

                    self.weight_map.clear()
                    for _, row in table.iterrows():
                        full_name = row["Bank Name"].upper().replace(" LTD", "").strip()
                        symbol = alias_map.get(full_name, None)
                        if symbol:
                            self.weight_map[symbol] = row["Weightage (%)"]
                    self.last_weightage_fetch = datetime.now().date()
                    return
            print("Weightage table not found")
        except Exception as e:
            print("Error fetching weightage:", e)

    def initialize_excel(self):
        excel = win32com.client.Dispatch("Excel.Application")
        excel.Visible = True

        if not os.path.exists(self.file_path):
            wb = excel.Workbooks.Add()
            sheet = wb.Worksheets(1)
            headers = ["Bank", "LTP", "Volume", "Change %", "Status", "Weightage"]
            for col, header in enumerate(headers, 1):
                sheet.Cells(1 + self.offset_rows, col).Value = header
            wb.SaveAs(self.file_path)
            return wb, sheet

        for wb in excel.Workbooks:
            if wb.FullName.lower() == self.file_path.lower():
                return wb, wb.Worksheets(1)
        try:
            wb = excel.Workbooks.Open(self.file_path)
            return wb, wb.Worksheets(1)
        except:
            wb = excel.Workbooks.Add()
            sheet = wb.Worksheets(1)
            headers = ["Bank", "LTP", "Volume", "Change %", "Status", "Weightage"]
            for col, header in enumerate(headers, 1):
                sheet.Cells(1 + self.offset_rows, col).Value = header
            wb.SaveAs(self.file_path)
            return wb, sheet

    def get_status(self, change):
        if change > 0:
            return "Buying"
        elif change < 0:
            return "Selling"
        return "Neutral"

    def update_excel(self):
        def get_last_trading_day(current_date):
            last_trading_day = current_date - timedelta(days=1)
            if current_date.weekday() == 0:
                last_trading_day = current_date - timedelta(days=3)
            elif current_date.weekday() == 6:
                last_trading_day = current_date - timedelta(days=2)
            elif current_date.weekday() == 5:
                last_trading_day = current_date - timedelta(days=1)
            return last_trading_day

        now = datetime.now()
        today_open = now.replace(hour=9, minute=15, second=0, microsecond=0)
        market_close = now.replace(hour=15, minute=30, second=0, microsecond=0)

        if now.weekday() >= 5:
            print("⚠️ Market is closed on Saturday and Sunday.")
            last_trading_day = get_last_trading_day(now)
            from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
            to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
        else:
            if now < today_open:
                print("⚠️ Market has not opened yet. Try again after 9:15 AM.")
                last_trading_day = get_last_trading_day(now)
                from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
                to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
            elif now > market_close:
                print("⚠️ Market is closed for the day.")
                from_date = (market_close - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
                to_date = market_close.strftime("%Y-%m-%d %H:%M")
            else:
                from_date = (now - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
                to_date = now.strftime("%Y-%m-%d %H:%M")

        self.get_banknifty_weightage()
        self.sheet.UsedRange.Clear()
        headers = ["Bank", "LTP", "Volume", "Change %", "Status", "Weightage"]
        for col, header in enumerate(headers, 1):
            self.sheet.Cells(1 + self.offset_rows, col).Value = header

        row_index = 2 + self.offset_rows
        for stock in self.bank_nifty_stocks:
            try:
                histParam = {
                    "exchange": "NSE",
                    "symboltoken": self.symbol_token_map[stock],
                    "interval": "ONE_MINUTE",
                    "fromdate": from_date,
                    "todate": to_date
                }
                response = self.smartApi.getCandleData(histParam)
                if response['status'] and response['data']:
                    candle = response['data'][-1]
                    ltp = candle[4]
                    volume = candle[5]
                    change_percent = ((candle[4] - candle[1]) / candle[1]) * 100
                    status = self.get_status(change_percent)
                    weight = self.weight_map.get(stock.upper(), "")
                    self.sheet.Cells(row_index, 1).Value = stock
                    self.sheet.Cells(row_index, 2).Value = ltp
                    self.sheet.Cells(row_index, 3).Value = volume
                    self.sheet.Cells(row_index, 4).Value = round(change_percent, 2)
                    self.sheet.Cells(row_index, 5).Value = status
                    self.sheet.Cells(row_index, 6).Value = weight
                    row_index += 1
                else:
                    print(f"⚠️ Failed to get {stock} ONE_MINUTE data:", response)
            except Exception as e:
                print(f"❌ Error fetching {stock} data:", e)

        self.sheet.Cells(row_index + 1, 1).Value = "Data Fetched At"
        self.sheet.Cells(row_index + 1, 2).Value = to_date

        self.wb.Save()
        print(f"✅ Bank Nifty data updated in Excel at {datetime.now().strftime('%H:%M:%S')}")

    def run(self, interval_sec=60):
        try:
            while True:
                self.update_excel()
                time.sleep(interval_sec)
        except KeyboardInterrupt:
            print("Stopped by user.")
        except Exception as e:
            print("Unexpected error:", e)


In [ ]:
# Usage Example:
dashboard = BankNiftyDashboard(
     api_key='FYBFTPOl',
     username='AAAF620196',
     pwd='0054',
     token='YS4MOR7OUZEHGNV3VFOQ5TVMRE',
     file_path=r"C:\Users\jaiad\Pro T\Final_draft1\banknifty_dashboard.xlsx",
     offset_rows=15  # Move table down by 5 rows
 )
dashboard.run(interval_sec=60)

[I 250618 09:56:23 smartConnect:124] in pool


✅ Bank Nifty data updated in Excel at 09:56:36
